In [1]:
# Libraries Installation
!pip install openai
# Required Libraries
import openai
import json
import os
from collections import OrderedDict


In [2]:
# Export your API Key to environment variable
# Upload the .env file to the directory "/content/"
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

False

In [27]:
import openai

# Set the API key for OpenAI
openai.api_key = "open_ai_key"

In [28]:
# Defining a class called RequirementProcessor
class RequirementProcessor:
    def __init__(self):
        # During initialization, we set the model to a specific GPT model name.
        self.model = "gpt-3.5-turbo-16k"

    def process_requirement(self, requirement_description, system_prompt):
        """
        Generates a response based on the given requirement description and system prompt using the OpenAI API.

        Parameters:
        - requirement_description (str): The description of the requirement.
        - system_prompt (str): The initial system prompt for context.

        Returns:
        - str: The generated text based on the provided input.
        """

        # Create a list of messages to send to the OpenAI API.
        # The first message is from the system for context and the second is the user's requirement.
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": requirement_description},
        ]

        # Set the temperature (affects randomness of output) and maximum token limit for the response.
        temperature = 1
        max_tokens = 2000

        # Call the OpenAI API's ChatCompletion method with the defined model and parameters.
        response = openai.Completion.create(
            model=self.model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )

        # Extract and strip the generated content from the API response.
        generated_text = response["choices"][0].message["content"].strip()

        # Return the processed/generated text.
        return generated_text


In [29]:
processor = RequirementProcessor()
position_name = input("What is the name of the position? ")

print("[Analyse JD] Important descriptions for the job (Enter 'STOP' when you are done):")
desc_content = []
while True:
    line = input()
    if line.strip() == 'STOP' or line.strip() == 'stop':
        break
    desc_content.append(line)
desc_content = '\n'.join(desc_content)
print("[Analyse JD] Description uploaded")

client_name = input("Name of the client: ")
min_exp = input("Minimum experience in years: ")
max_exp = input("Maximum experience in years: ")
min_budget = input("Minimum salary budget (p.a): ")
max_budget = input("Maximum salary budget (p.a): ")
location_info = input("Location information: ")
notice_period = input("Notice period: ")
mandatory_skills = input("Enter the mandatory or must have skills for the job requirement: ")
requirement_description = f'''Position name: {position_name}, description of the job and requirements: {desc_content}, Name of the client: {client_name}, \
Minimum experience required: {min_exp}, Maximum experience required: {max_exp}, Minimum budget for the salart p.a: {min_budget}, Maximum budget for the salart p.a: {max_budget}\
Location of the job: {location_info}, Notice period: {notice_period}, Mandatory or Must have skills required for this job: {mandatory_skills}'''

[Analyse JD] Important descriptions for the job (Enter 'STOP' when you are done):
[Analyse JD] Description uploaded


In [30]:
system_prompt_v1 = '''Give information about year of experience, CTC, notice period and educational requirement if available in the description. \
If not mark it as none. Also, summarize the technical skills needed.\
Summarize other needed skills like soft skills. \
Also, add must have skills - these are the most important requirements from the job description, clearly mentioned as must have skills. \
location is the place where the person will be posted. \
'''

extracted_info = processor.process_requirement(requirement_description, system_prompt_v1)
print("extracted information", extracted_info)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
import json

def save_to_json(data, filename='data.json'):
    """
    Save a Python data structure to a JSON file.

    Args:
    - data (dict or str): The Python data structure to be saved. Can be a string (that can be loaded as JSON) or a dictionary.
    - filename (str): The name of the JSON file.

    Returns:
    - None
    """
    # Check if the data is already a string and try to load it into a Python object.
    # If it's already a Python object (like a dictionary or list), then pass.
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except json.JSONDecodeError:
            raise ValueError("The provided string is not valid JSON.")
    elif not isinstance(data, (dict, list)):
        raise TypeError("The data should either be a valid JSON string, dictionary, or list.")

    with open(filename, 'w') as f:
        json.dump(data, f)

save_to_json(extracted_info, '/content/requirements_output.json')  # This will save the data to 'my_data.json' file.


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import requests
# https://drive.google.com/file/d/17V_o0Snt-Lj0FmegENPQ_rXpvWTWlZgQ/view?usp=sharing
def download_file_from_google_drive(file_id, destination):
    base_url = "https://drive.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(base_url, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(base_url, params=params, stream=True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)
# Example Usage
# file_id = '1HaM3IeK2-iqyZzeQmCnAzKLcF9NF-mSo'  # Replace with your file's ID
# destination = 'resume_data.zip'
file_id = '17V_o0Snt-Lj0FmegENPQ_rXpvWTWlZgQ'
destination = 'Webinar_resumes.zip'  # Replace with your desired file name and extension
download_file_from_google_drive(file_id, destination)

In [ ]:

class ResumeProcessor:
    def __init__(self):
        self.resume_counter = 1
    # Function to extract the CTC bounds from a given string in the format "XX-YY".
    def get_ctc_bounds(self, ctc_range):
        messages = [
            {"role": "system", "content": "You are an assistant to a recruiter. You will be given a range consisting of lower and upper salary amount in dollars. \
            Return the upper and lower values in dollars in expanded form. No currency should be mentioned. Output should be in JSON like {'lower':lower salary, 'upper': upper salary}"},
            {"role": "user", "content": f"The salary range is {ctc_range}."},
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=1000
        )
        generated_texts = [choice.message["content"].strip() for choice in response["choices"]]
        # print("iniital output", generated_texts)

        pattern = r'(\d{1,3}(?:,\d{3})*)\s*(dollars)?'

        matches = re.findall(pattern, generated_texts[0])
        # print("matches", matches)
        if matches and len(matches) == 2:


            # Extracting lower and upper salary from the matches and removing commas
            try:
              lower_salary = float(matches[0].replace(',', ''))
              upper_salary = float(matches[1].replace(',', ''))
            except:
              lower_salary = float(matches[0][0].replace(',', ''))
              upper_salary = float(matches[1][0].replace(',', ''))

            return lower_salary, upper_salary
        else:
            return None
        return generated_texts[0]

    def convert_notice_period_to_days(self, jd_notice):
        messages = [
            {"role": "system", "content": "You're assisting a recruiter. Convert the provided notice period into days. \
            1 month is typically 30 days, and 1 year is 365 days. The final output should be 'Days': Number of days."},
            {"role": "user", "content": f"Given notice period: {jd_notice}. Return the output in json with the field 'Days'"}
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=1000
        )
        generated_texts = [choice.message["content"].strip() for choice in response["choices"]]
        # print(generated_texts, generated_texts[0])
        return int(re.findall(r'\d+', generated_texts[0])[0])
    def extract_and_rename(self, zip_file_path):
        # Specify the directory where the files will be extracted.
        extract_path = "extracted_files"

        # Open the zip file for reading.
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all files/directories in the zip to the specified directory.
            zip_ref.extractall(extract_path)

        # Start by assuming the path for the resumes is the extraction directory.
        resume_path = extract_path

        # Loop over each item (file or directory) in the extraction directory.
        for item in os.listdir(extract_path):
            # Construct the full path for the item.
            item_path = os.path.join(extract_path, item)

            # Check if the current item is a directory and if its name contains spaces.
            if os.path.isdir(item_path) and ' ' in item:
                # Replace spaces in the directory name with underscores.
                new_name = item.replace(' ', '_')
                # Construct the new path for the directory after renaming.
                new_path = os.path.join(extract_path, new_name)

                # If a directory with the new name doesn't already exist, create one.
                if not os.path.exists(new_path):
                    os.makedirs(new_path)

                # Copy each file/sub-item from the old directory (with spaces in the name) to the new directory.
                for sub_item in os.listdir(item_path):
                    shutil.copy2(os.path.join(item_path, sub_item), new_path)

                # Remove the old directory (with spaces in the name).
                shutil.rmtree(item_path)
                # Update the resume path to point to the new directory.
                resume_path = new_path
            else:
                # If the item is not a directory (i.e., it's a file), update the resume path to point to this file.
                resume_path = item_path

        # Return the path where the resumes are located (either a directory or a single file).
        return resume_path

    def generate_random_data(self, upper_bound, lower_bound, total_resumes, max_notice_period_days):
        # Define a list of Californian cities.
        californian_cities = ['San Francisco', 'San Diego', 'Sacramento', 'Oakland']

        # Modify thresholds to skew random generation towards more favorable candidates.
        threshold_80_percent = 0.80 * total_resumes  # Increased from 50%
        threshold_90_percent = 0.90 * total_resumes
        threshold_10_percent = 0.10 * total_resumes  # Increased from 20%

        # Skew CTC generation to be within the desired range for 80% of candidates.
        if self.resume_counter < threshold_80_percent:
            current_ctc = round(random.uniform(lower_bound, upper_bound), 1)
            expected_ctc = round(random.uniform(current_ctc + 1, upper_bound), 1)
        else:
            current_ctc = round(random.uniform(0.5 * lower_bound, lower_bound - 1), 1)
            expected_ctc = round(random.uniform(current_ctc + 1, upper_bound), 1)

        # 90% candidates are willing to relocate.
        if self.resume_counter < threshold_90_percent:
            willing_to_relocate = "yes"
        else:
            willing_to_relocate = "no"

        # Adjust notice period distribution.
        if self.resume_counter < threshold_10_percent:
            notice_period = f"{random.randint(max_notice_period_days + 1, max_notice_period_days + 30)} days"
        else:
          notice_period = f"{random.randint(10, max_notice_period_days//2)} days"

        current_location = random.choice(californian_cities)
        self.resume_counter += 1

        return current_ctc, expected_ctc, willing_to_relocate, current_location, notice_period

    def random_data_resumes(self, zip_file_path):
        # Extract resumes from the zip file and rename them if necessary.
        resume_path = self.extract_and_rename(zip_file_path)

        # List out all the resume files present in the extracted path with specific extensions (.pdf, .doc, .docx).
        resume_files = [f for f in os.listdir(resume_path) if f.endswith(('.pdf', '.doc', '.docx'))]
        applications = []

        # Read the job requirements from the "requirements_output.json" file.
        with open("requirements_output.json", "r") as f:
            job_req = json.load(f)
        # Convert the notice period (from job requirements) to days.
        notice_period_criteria = self.convert_notice_period_to_days(job_req.get("notice_period", ""))
        # Get the lower and upper bounds of the CTC from the job requirements.
        print(self.get_ctc_bounds(job_req.get("CTC", "")))
        lower_bound, upper_bound = self.get_ctc_bounds(job_req.get("CTC", ""))

        # Iterate over each resume file.
        for filename in resume_files:
            resume_file_path = os.path.join(resume_path, filename)
            # Generate an email ID using the filename (assuming the file name does not contain periods other than the file extension).
            email_id = filename.split('.')[0] + "@example.com"
            # Generate random data for the current resume.
            current_ctc, expected_ctc, willing_to_relocate, current_location, notice_period = self.generate_random_data(upper_bound, lower_bound, len(resume_files), notice_period_criteria)

            # Append the generated data for the current resume to the applications list.
            applications.append({
                'current_ctc': current_ctc,
                'expected_ctc': expected_ctc,
                'willing_to_relocate': willing_to_relocate,
                'current_location': current_location,
                'notice_period': notice_period,
                'email_id': email_id,
                'resume_path': filename
            })

        # Save the entire applications list to a JSON file.
        with open("all_applications.json", "w") as f:
            json.dump(applications, f, indent=4)

# Using the class:
processor = ResumeProcessor()
processor.random_data_resumes("/content/Webinar_resumes.zip")

In [ ]:
# Testing various formats for CTC:
result = processor.get_ctc_bounds("100k-150k")
print(f"Input '100k-150k', Output: {result} \n")



In [ ]:
# Testing various formats for Notice period:
result = processor.convert_notice_period_to_days("1 month")
print(f"Input '1 month', Output: {result} \n")


In [ ]:

class FilterCTC:
    def __init__(self):
        # Create an instance of the resume processor class
        self.processor = ResumeProcessor()

    def get_ctc_check(self, budget_min, budget_max, cr_ctc, exp_ctc):
        """
        Check if the candidate's current and expected CTC are within the company's budget range.

        :param budget_min: Minimum budget of the company for CTC
        :param budget_max: Maximum budget of the company for CTC
        :param cr_ctc: Candidate's current CTC
        :param exp_ctc: Candidate's expected CTC

        :return: True if candidate's CTC is within budget, else False
        """
        messages = [
            {"role": "system", "content": "You are an assistant to a recruiter. You will be given the budget of the company (a range), candidate's current total compensation, and expected total compensation. \
            Return 'yes' if the current compensation is greater than the budget minimum and the expected total compensation is less than the maximum budget. Both conditions should be met for a 'yes'. For all other cases, return 'no'."},
            {"role": "user", "content": f"Company budget minimum: {budget_min}, company budget maximum: {budget_max}, candidate current total compensation: {cr_ctc}, candidate total expected compensation: {exp_ctc}"},
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=1000
        )
        result = response.choices[0].message["content"].strip().lower()
        return result == "yes"

    def filter_CTC_resumes(self):
        """
        Load resumes and job requirements, and filter out resumes based on CTC.
        The filtered resumes are saved to 'filtered_applications_ctc.json' and the ones which didn't meet criteria to 'removed_resume_ctc.json'.
        """
        # Load all applications from the JSON file
        with open("all_applications.json", "r") as f:
            applications = json.load(f)

        # Load job requirements from the JSON file
        with open("requirements_output.json", "r") as f:
            job_req = json.load(f)

        # Assume a method exists to get the CTC bounds from the job requirements
        lower_bound, upper_bound = self.processor.get_ctc_bounds(job_req.get("CTC", ""))

        # Filter the applications based on the current and expected CTC criteria
        filtered_applications_ctc = [app for app in applications if self.get_ctc_check(lower_bound, upper_bound, app['current_ctc'], app['expected_ctc'])]

        # Identify the applications that were removed due to not meeting the CTC criteria
        removed_due_to_ctc = [app for app in applications if app not in filtered_applications_ctc]

        # Save the filtered applications to a new JSON file
        with open("filtered_applications_ctc.json", "w") as f:
            json.dump(filtered_applications_ctc, f, indent=4)

        # Save the applications that didn't meet the criteria to a separate JSON file
        with open("removed_resume_ctc.json", "w") as f:
            json.dump(removed_due_to_ctc, f, indent=4)
filterctc = FilterCTC()
filterctc.filter_CTC_resumes()

In [ ]:
# Testing various formats for CTC check:
budget_min = "100k"
budget_max = "150k"
cr_ctc = "120,000"
exp_ctc = "140,000"
result = filterctc.get_ctc_check(budget_min, budget_max, cr_ctc, exp_ctc)
print(f"Input '{budget_min, budget_max, cr_ctc, exp_ctc}', Output: {result} \n")


In [ ]:

class FilterCity:
    # Initialize the object with job requirements and filtered ctc applications
    def __init__(self, job_requirements_path, filtered_ctc_applications_path):
        # Load the job requirements from the given file
        with open(job_requirements_path, "r") as f:
            self.job_req = json.load(f)

        # Load the previously filtered applications from the given file
        with open(filtered_ctc_applications_path, "r") as f:
            self.filtered_applications_ctc = json.load(f)

    def check_city(self, current_city, job_city, drive_hour = "1-hour"):
        messages = [
            {"role": "system", "content": f"You're assisting a recruiter. \
            Determine if the candidate's current city ({current_city}) and the prospective job city ({job_city})\
            are the same (even if the names vary or there may be spelling mistakes) then return 'yes'.\
            If both the cities {current_city} & {job_city} are within a {drive_hour} drive by car from each other, in that case you should \
            return 'yes', otherwise return 'no'."},
            {"role": "user", "content": f"Candidate's current city: {current_city}. Job's city: {job_city}."}
        ]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=100
        )
        generated_texts = [choice.message["content"].strip() for choice in response["choices"]]
        return "Yes" in generated_texts or "yes" in generated_texts
    # Filter applicants based on their willingness to relocate or if they are in the same city
    def filter_by_willing_to_relocate_and_city(self):
        filtered_applications = []
        job_city = self.job_req.get("City", "")

        # Iterate through each application
        for application in self.filtered_applications_ctc:
            # If the 'willing_to_relocate' field is 'n.a.', set it to 'yes'
            if application['willing_to_relocate'].lower() == "n.a.":
                application['willing_to_relocate'] = 'yes'

            # Check if the applicant's current location matches the job city
            same_city = application['current_location'].lower() == job_city.lower()

            # Check if the applicant is willing to relocate
            willing_to_relocate = application['willing_to_relocate'].lower() == 'yes'

            # If applicant is in the same city or willing to relocate, append to the filtered list
            if same_city or willing_to_relocate:
                filtered_applications.append(application)
                continue
            # If not in the same city and not willing to relocate, check willingness using the GPT model
            if not same_city and not willing_to_relocate:
                if self.check_city(application['current_location'], job_city):
                    filtered_applications.append(application)

        return filtered_applications

    # Save the filtered and removed applications to separate files
    def save_filtered_and_removed(self, filtered_applications):
        # Find the applications that were removed due to city constraints
        removed_due_to_city = [app for app in self.filtered_applications_ctc if app not in filtered_applications]

        # Save the filtered applications
        with open("filtered_applications_city.json", "w") as f:
            json.dump(filtered_applications, f, indent=4)

        # Save the removed applications
        with open("removed_due_to_city.json", "w") as f:
            json.dump(removed_due_to_city, f, indent=4)

    # Process the filtering and saving operations
    def process_filtering(self):
        filtered_applications = self.filter_by_willing_to_relocate_and_city()
        self.save_filtered_and_removed(filtered_applications)

# Create an instance of the ApplicantFilter and process the applications
applicant_filter = FilterCity("requirements_output.json", "filtered_applications_ctc.json")
applicant_filter.process_filtering()

In [ ]:
# Testing our code with different variations:
current_city = "San Francisco"
job_city = "san francisco"
drive_distnace = "1-hour"
result = applicant_filter.check_city(current_city, job_city, drive_distnace)
print(f"Applicant location: {current_city}, Job locatoion: {job_city}, Drive distnace: {drive_distnace}")
print(f"Result: {result} \n")



In [ ]:

class FilterNotice:

    def __init__(self):
        # Initialize the ResumeFilter class with an instance of ResumeProcessor class
        self.processor = ResumeProcessor()

    def check_notice(self, jd_notice, can_notice):
        """
        Convert notice periods given in months, years, or days to days, and check if the candidate's
        notice period is less than or equal to the job's notice period.

        :param jd_notice: Job's notice period
        :param can_notice: Candidate's notice period

        :return: 'yes' or 'no' indicating if the candidate's notice period is less than or equal to the job's notice period
        """
        jd_notice = self.processor.convert_notice_period_to_days(jd_notice)
        can_notice = self.processor.convert_notice_period_to_days(can_notice)
        messages = [
            {"role": "system", "content": "You're assisting a recruiter. Convert the provided notice periods into days. \
            1 month is typically 30 days, and 1 year is 365 days. If the candidate's notice period in days is less than the job's notice period in days, then return 'yes'. \
            Otherwise, reply 'no'. The output must be 'yes' or 'no'"},
            {"role": "user", "content": f"Job's notice period: {jd_notice}. Candidate's notice period: {can_notice}."}
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            max_tokens=100
        )
        generated_texts = [choice.message["content"].strip().lower() for choice in response["choices"]]
        # print("initial output", generated_texts, jd_notice, can_notice, "jd_notice, can_notice")
        return generated_texts[0]

    def filter_and_save(self):
        # Load the job requirements
        with open("requirements_output.json", "r") as f:
            job_req = json.load(f)

        # Get the job's notice period criteria
        notice_period_criteria = job_req.get("notice_period", "")

        # Load the previously filtered applications based on city
        with open("filtered_applications_city.json", "r") as f:
            filtered_applications_city = json.load(f)

        # Filter the previously filtered applications based on the notice period
        filtered_applications_notice = [app for app in filtered_applications_city if self.check_notice(notice_period_criteria, app['notice_period']) == 'yes']
        removed_due_to_notice = [app for app in filtered_applications_city if app not in filtered_applications_notice]

        # Save the final filtered applications to a new JSON file
        with open("filtered_applications.json", "w") as f:
            json.dump(filtered_applications_notice, f, indent=4)

        # Save the resumes that were removed in the final filtering to another JSON file
        with open("removed_resume_final.json", "w") as f:
            json.dump(removed_due_to_notice, f, indent=4)

# Assuming the ResumeProcessor class is defined elsewhere, we create an instance of it

filterer = FilterNotice()
filterer.filter_and_save()

In [ ]:
# Testing various cases:
notice_period_jd = "One Month"
notice_period_candidate = "29 days"
result = filterer.check_notice(notice_period_jd, notice_period_candidate)
print(f"Output for JD Notice period: {notice_period_jd} & Candidate notice period: {notice_period_candidate} is {result} \n")


In [ ]:

# Define a function to count and return the number of entries in a given JSON file.
def count_entries_in_json(json_filepath):
    # Open and read the content of the JSON file.
    with open(json_filepath, "r") as f:
        data = json.load(f)  # Parse and load the JSON content into a variable.
        return len(data)  # Return the number of top-level entries in the loaded data.

# A list of file paths containing JSON data that we want to process.
files = [
    "/content/all_applications.json",
    "/content/filtered_applications.json",
    "/content/removed_due_to_city.json",
    "/content/removed_resume_ctc.json",
    "/content/removed_resume_final.json"
]

# Loop through each file in the list, count its entries, and print the result.
for file in files:
    count = count_entries_in_json(file)  # Use the function to count entries for the current file.
    print(f"{file}: {count} entries")  # Display the file path along with its count of entries.

In [ ]:
from google.colab import files

# List of file paths that you want to download
file_paths = [
    "/content/filtered_applications.json",
    "/content/all_applications.json"

]

# Download each file to your local system
for path in file_paths:
    files.download(path)

In [ ]:

def initial_score(text, prompt):
    """
    Use OpenAI's model to generate a response based on the given prompt and text.

    Args:
    - text (str): The content to be processed.
    - prompt (str): The instruction or question for the model to guide its response.

    Returns:
    - str: Model-generated text based on the input text and prompt.
    """

    # Specify the model and token limits
    model = "gpt-3.5-turbo-16k"
    max_tokens = 2000

    # Create a list of messages to simulate a conversation with the model.
    # The system starts with a prompt, and the user provides the input text.
    messages = [
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": text},
        ]

    # Make a request to the OpenAI API for the generated response.
    response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=1,
            max_tokens=max_tokens
        )

    # Extract the generated text from the model's response
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]

    return generated_texts[0]


In [ ]:

def user_select_number_of_resumes(total_resumes, default=2):
    """
    Allow the user to input a number of resumes to process.
    If no input is given, the default value is returned.

    Args:
    - total_resumes (int): Total number of resumes available.
    - default (int): The default number to return if no input.

    Returns:
    - int: The number of resumes the user wants to process.
    """
    print(f"Total resumes available: {total_resumes}")
    user_input = input(f"How many resumes do you want to process? (Default is {default}): ")

    # If the user doesn't provide any input, return the default value.
    if not user_input:
        return default

    try:
        # Convert user input to an integer and ensure it's within the range.
        selected_num = int(user_input)
        if 1 <= selected_num <= total_resumes:
            return selected_num
        else:
            print(f"Please select a number between 1 and {total_resumes}.")
            return user_select_number_of_resumes(total_resumes, default)
    except ValueError:
        # If the user provides non-numeric input, prompt them again.
        print("Please enter a valid number.")
        return user_select_number_of_resumes(total_resumes, default)

# Read the filtered_applications_summary data from the JSON file
json_data = read_json('/content/filtered_applications_summary.json')

# Display total resumes and get the user's choice
n = user_select_number_of_resumes(len(json_data))

# Randomly select n resumes
selected_applications = random.sample(json_data, n)

In [ ]:
prompt_version_1 = f'''You are an assistant to a recruiter. Your job is to evaluate a resume for a particular skill. The skills for which you need to do
evaluation are {must_have_skills}. You need to find the projects in which a particular skill from this list has been applied.'''

for application in selected_applications:
    if 'resume_path' in application and 'email_id' in application:

        # Extract resume text
        resume_text = read_document(os.path.join(resume_path, application['resume_path']))
        resume_text, _, _ = check_and_trim(resume_text)

        # Directly assign the resume_summary without json.loads()
        resume_summary = application['resume_summary']

        # Score the resume against job requirements
        initial_score_output = initial_score(resume_text, prompt_version_1)

        print(f'''[Matching Request] for {resume_summary["name_of_candidate"]} ''', initial_score_output)

In [ ]:
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v1 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". \
    Now generate a criterion to score each of the skill between 0 and 5.'''

    messages = [
            {"role": "system", "content": f"{prompt_v1}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]



criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)




In [ ]:
string_match = """
skill1:
No projects: 0
1 project or mention: one or two
2 or 3 projects or mentions: three or four
more than or equal to five projects or mentions: five
"""
def criterion_gen(must_have):
    model="gpt-3.5-turbo-16k"
    max_tokens=4000
    prompt_v2 = f'''I have a resume. I want to find a person whose resume has skills in {must_have}. \
    Let's think step by step on how to look into a resume and give score based on each of the skill mentioned in {must_have}. \
    As an example, take the case where the must have skills are "SKILL1, SKILL2, SKILL3, SKILL4, SKILL5". Now score each of the skill between 0 and 5.\
    For example use the criterion given in {string_match}'''

    messages = [
            {"role": "system", "content": f"{prompt_v2}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]

criterion_gen_output = criterion_gen(must_have_skills)
print(f'''[Criterion Generation Request] ''', criterion_gen_output)

In [ ]:
def final_score_func(resume_text, must_have_skills):
    # Read the criteria and string_match from the text file
    criterion_gen_output, string_match = read_from_textfile("/content/criterion_and_string_match_output.txt")
    prompt=f'''You are an assistant to a recruiter. \
    Use the criteria given by {criterion_gen_output} to judge the resume given to you and give score to \
    each of the skills present in {must_have_skills}.'''

    # Assuming the function score_from_criterion exists and takes these parameters
    score_output = score_from_criterion(prompt, resume_text, criterion_gen_output, must_have_skills, string_match)

    return score_output

for application in selected_applications:
    if 'resume_path' in application and 'email_id' in application:

        # Extract resume text
        resume_text = read_document(os.path.join(resume_path, application['resume_path']))
        resume_text, _, _ = check_and_trim(resume_text)

        # Directly assign the resume_summary without json.loads()
        resume_summary = application['resume_summary']

        final_score_out = final_score_func(resume_text, must_have_skills)
        print(f'''[Score Request] for {resume_summary["name_of_candidate"]} ''', final_score_out)
